In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
 #%%

# google colab configurations 
# these might change

# torch imports
import torch

# import utils
import json
from datetime import date
import os

google_colab = input('Running on Colab?\n')
if google_colab=='yes' : 

    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)


    COLAB_CONFIG = '1o5nZNVmNBq3UQRxwYIwOATX0c0GyHPRN'
    COLAB_CONFIG_FILE ='colab_config.json'

    colab_json = drive.CreateFile({'id': COLAB_CONFIG})
    colab_json.GetContentFile(COLAB_CONFIG_FILE)

    with open(COLAB_CONFIG_FILE) as f:
        temp = f.read()
    colab_config = json.loads(temp)

    for key in colab_config:
        globals()[key] = drive.CreateFile({"id":colab_config[key]['id']})
        globals()[key].GetContentFile(colab_config[key]['file'])

    from trainer import RegressionTrainer
    from datageneration import DataGenerator
    from helpers import Helper, Plotter
    
else:
    import sys, os 
    BASE_DIR = '/'.join(os.getcwd().split('/')[:-1])
    sys.path.append(BASE_DIR)
    print(f'Top level directory is set to {BASE_DIR}')

    # custom imports
    from cnp.trainer import RegressionTrainer
    from cnp.datageneration import DataGenerator
    from cnp.helpers import Helper

Running on Colab?

Top level directory is set to /Users/frederik/Google Drive/University/UC3M/TFM/cnp_repo/neural-processes


In [3]:

# we first get the variables for the configuration file
os.environ["CHECKPOINT_DIR"] = "checkpoints"
os.environ["MODEL_PARAMS"] = "default"

CHECKPOINT_DIR = os.environ['CHECKPOINT_DIR']
MODEL_PARAMS = os.environ['MODEL_PARAMS']

if google_colab=='yes':
    config_dir = ''
else:
    config_dir = os.path.join(BASE_DIR,'configs')

with open(os.path.join(config_dir,'config.json')) as f:
    file = f.read()
    default_conf = json.loads(file)



In [4]:
CONFIGURATION='paper_configuration'

In [5]:
# checking whether CUDA is available 
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
    default_conf[CONFIGURATION]['experiment_params']['train_on_gpu'] = True
else: 
    print('No GPU available, training on CPU')
    default_conf[CONFIGURATION]['experiment_params']['train_on_gpu'] = False

No GPU available, training on CPU


## Generating a dataset

In [6]:
# creating an instance of to orchestrate the training
trainer = RegressionTrainer(**default_conf[CONFIGURATION]['experiment_params'])

In [7]:
model_weights = trainer.run_training(**default_conf[CONFIGURATION]['data_gen_params'], valiloader=None, plotting=True)

TypeError: run_training() got an unexpected keyword argument 'trainloader'

In [8]:
default_conf[CONFIGURATION]['data_gen_params']

{'num_instances_train': 64,
 'num_instances_vali': 1,
 'noise': 0.0004,
 'length_scale': 0.4,
 'gamma': 1,
 'batch_size_train': 64,
 'batch_size_vali': 1,
 'trainloader': None}

In [22]:
default_conf[CONFIGURATION]

{'experiment_params': {'n_epochs': 200000,
  'lr': 0.0001,
  'min_funcs': 2,
  'max_funcs': 10,
  'max_contx': 10,
  'min_contx': 3,
  'dim_observation': 400,
  'dimx': 1,
  'dimy': 1,
  'dimr': 128,
  'dimout': 2,
  'dropout': 0,
  'num_layers_encoder': 4,
  'num_neurons_encoder': 128,
  'num_layers_decoder': 3,
  'num_neurons_decoder': 128,
  'train_on_gpu': False,
  'print_after': 5000,
  'generatedata': True,
  'range_x': [-2, 2]},
 'data_gen_params': {'num_instances_train': 64,
  'num_instances_vali': 1,
  'noise': 0.0004,
  'length_scale': 0.4,
  'gamma': 1,
  'batch_size_train': 64,
  'batch_size_vali': 1,
  'trainloader': None}}